In [0]:
def get_processed_srs(config):
  
  """

  Reads raw Service Requests data, and process it to make it ready for joining with the master table.

  Parameters:
  config: config dictionary as defined in get_config.

  Returns:
  Processed Service Requests data.

  """

  sr = get_raw_df(config, 'service_requests')
  sr['id_date'] = sr['dtupdatedt'] + pd.offsets.MonthBegin(1)
  assert sr.id_date.isna().mean() < .01, 'There is more than 1% NAs in the update date of the Service Requests table'
  sr = sr.loc[~sr.id_date.isna()]

  id_dates_df = sr[['id_date']].drop_duplicates().assign(k = 1)
  processed_srs = sr[['htenant', 'hunit']].drop_duplicates().assign(k = 1)
  processed_srs = pd.merge(processed_srs, id_dates_df, on = 'k', how = 'outer').drop(columns = 'k')
  processed_srs.sort_values(['htenant', 'hunit', 'id_date'], inplace = True)
  assert processed_srs.isna().sum().sum() == 0

  id_cols = ['htenant', 'hunit', 'id_date']
  id_df = sr[id_cols].drop_duplicates()
  agg_df = sr.groupby(id_cols)['hmy'].count().reset_index().rename(columns = {'hmy': 'n_SRs'})
  id_df = pd.merge(id_df, agg_df, on = id_cols, how = 'left')
  id_df['n_SRs'].fillna(0, inplace = True)


  categories = {
  'ssubcat': {
    'term': 'termination_enquiry',
    'change': 'change_of_unit',
    'cfa': 'common_facilities_amenities',
    'noise': 'complaint_noise'
  },
  'scategory': {
    'ex_lease': 'existing_leasing_enquiry',
    'common_area': 'pm_common_area',
    'enq_lease': 'leasing_enquiry'
  },
  'spriority': {
    'immediate': 'immediate',
    'major': 'major'
  }
  }

  def add_custom_count(id_df, sr, id_cols, categories, j, i):
    new_col_name = 'n_SRs_' + j + '_' + i
    agg_df = sr.loc[sr[j] == categories[j][i]].groupby(id_cols)['hmy'].count().reset_index().rename(columns = {'hmy': new_col_name})
    id_df = pd.merge(id_df, agg_df, on = id_cols, how = 'left')
    id_df[new_col_name].fillna(0, inplace = True)
    return id_df

  for j in categories.keys():
    for i in categories[j].keys():
      id_df = add_custom_count(id_df, sr, id_cols, categories, j, i)


  processed_srs = pd.merge(processed_srs, id_df, on = id_cols, how = 'left').fillna(0)

  processed_srs['n_avg_SRs_3m'] = processed_srs.groupby(['htenant', 'hunit'])['n_SRs'].rolling(3).mean().values
  processed_srs['n_avg_SRs_6m'] = processed_srs.groupby(['htenant', 'hunit'])['n_SRs'].rolling(6).mean().values

  for j in categories.keys():
    for i in categories[j].keys():
      new_col_name = 'n_SRs_' + j + '_' + i
      processed_srs[new_col_name + '_3m'] = processed_srs.groupby(['htenant', 'hunit'])[new_col_name].rolling(3).mean().values
      processed_srs[new_col_name + '_6m'] = processed_srs.groupby(['htenant', 'hunit'])[new_col_name].rolling(6).mean().values

  processed_srs.fillna(0, inplace = True)

  return processed_srs